<a href="https://colab.research.google.com/github/Fazna-kozhipparambil/Smart-job-genie-Autonomous-Job-Search-Application-AI-Agent/blob/main/SmartJob_Genie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.Install Packages**

In [ ]:
!pip install --upgrade google-genai google-generativeai requests python-docx --quiet

print("Packages installed/upgraded.")


**2.Gemini Setup**

In [ ]:
import os
import json
import uuid
import requests
from pathlib import Path
from docx import Document

# Set Gemini API key
os.environ['GEMINI_API_KEY'] = " Gemini_API_key"#YOUR_GEMINI_KEY_HERE

# Import Gemini client
from google import genai
client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])

# Test client
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words"
)
print("Gemini Response:", response.text)


**3.MCP Tools: Memory, Files, Notifications**

In [ ]:
ARTIFACTS_DIR = Path("artifacts")
ARTIFACTS_DIR.mkdir(exist_ok=True)

class MemoryBank:
    def __init__(self, path="memory.json"):
        self.path = Path(path)
        if not self.path.exists():
            self.db = {'profiles': {}, 'jobs': {}, 'applications': {}, 'preferences': {}}
            self._flush()
        else:
            self.db = json.loads(self.path.read_text())
    def _flush(self): self.path.write_text(json.dumps(self.db, indent=2))
    def save_profile(self, user_id, profile): self.db['profiles'][user_id] = profile; self._flush()
    def get_profile(self, user_id): return self.db['profiles'].get(user_id)
    def save_jobs(self, jobs):
        for j in jobs: self.db['jobs'][j['id']] = j
        self._flush()
    def list_jobs(self): return list(self.db['jobs'].values())
    def log_application(self, app_rec):
        user_id = app_rec['user_id']
        self.db['applications'].setdefault(user_id, []).append(app_rec)
        self._flush()

class FileServer:
    def __init__(self, root=ARTIFACTS_DIR): self.root = Path(root)
    def save_text(self, filename, text):
        p = self.root / filename
        p.write_text(text, encoding='utf-8')
        return str(p)
    def list_artifacts(self): return [str(p) for p in self.root.iterdir()]

class Notifier:
    def notify(self, user_id, subject, message):
        print(f"[NOTIFY] user={user_id} subject={subject}\n{message[:300]}\n---")

memory = MemoryBank()
files = FileServer()
notifier = Notifier()
print("MCP tools ready. Memory path:", memory.path)


**4.Job API Wrapper**

In [ ]:
class JobAPI:
    def search_remotive(self, q, limit=10):
        url = "https://remotive.com/api/remote-jobs"
        try:
            r = requests.get(url, params={'search': q}, timeout=12)
            j = r.json()
            out = []
            for item in j.get('jobs', [])[:limit]:
                out.append({
                    'id': str(item.get('id')) or ('remotive-' + uuid.uuid4().hex),
                    'title': item.get('title'),
                    'company': item.get('company_name'),
                    'location': item.get('candidate_required_location'),
                    'description': item.get('description') or '',
                    'url': item.get('url'),
                    'posted_at': item.get('publication_date')
                })
            return out
        except: return []

    def search_arbeitnow(self, q, limit=10):
        url = "https://api.arbeitnow.com/v1/jobs"
        try:
            r = requests.get(url, timeout=12)
            j = r.json()
            out = []
            for item in j.get('data', []):
                if q.lower() in (item.get('title') or '').lower() or q.lower() in (item.get('description') or '').lower():
                    out.append({
                        'id': item.get('slug') or str(uuid.uuid4().hex),
                        'title': item.get('title'),
                        'company': item.get('company_name'),
                        'location': item.get('location'),
                        'description': item.get('description') or '',
                        'url': item.get('url'),
                        'posted_at': item.get('created_at')
                    })
                    if len(out) >= limit: break
            return out
        except: return []

    def search(self, q, limit=10):
        a = self.search_remotive(q, limit=limit)
        b = self.search_arbeitnow(q, limit=limit)
        seen = set(); merged=[]
        for job in a + b:
            if job.get('url') in seen: continue
            seen.add(job.get('url')); merged.append(job)
            if len(merged)>=limit: break
        return merged

job_api = JobAPI()
print("JobAPI ready.")


**5.Agents: Profile Analyzer**

In [ ]:
class ProfileAnalyzerAgent:
    def extract_profile_with_gemini(self, resume_text: str) -> dict:
        prompt = f"""
        Extract a JSON profile from this resume text:
        {resume_text}

        Output strictly in JSON format:
        {{
          "user_id": "string",
          "name": "string",
          "title": "string",
          "skills": ["string"],
          "experience_years": int,
          "summary": "string"
        }}
        """
        response = client.models.generate_content(model="gemini-2.5-flash", contents=prompt)
        try:
            profile_json = json.loads(response.text)
        except:
            profile_json = {"user_id":"demo","name":"Demo User","title":"Demo","skills":[],"experience_years":0,"summary":resume_text[:200]}
        return profile_json

    def receive(self, msg):
        if msg.get('action') == 'extract_profile':
            resume_text = msg.get('resume_text','')
            profile = self.extract_profile_with_gemini(resume_text)
            memory.save_profile(profile['user_id'], profile)
            return {'profile': profile}
        return {'error': 'unknown action'}

profile_agent = ProfileAnalyzerAgent()
print("ProfileAnalyzerAgent ready.")


**6.Job Search Agent**

In [ ]:
class JobSearchAgent:
    def receive(self, msg):
        if msg.get('action')=='search_jobs':
            query = msg.get('query','')
            limit = msg.get('limit',10)
            jobs = job_api.search(query, limit)
            memory.save_jobs(jobs)
            return {'jobs': jobs}
        return {'error': 'unknown action'}

search_agent = JobSearchAgent()
print("JobSearchAgent ready.")


**7.Matching & Ranking Agent**

In [ ]:
class MatchAgent:
    def score_with_gemini(self, profile, job):
        prompt = f"""
        Given user profile and job JSON, compute skill match (0-10) and reason.
        Profile: {json.dumps(profile)}
        Job: {json.dumps(job)}
        Output strictly as JSON: {{"score": float, "reason": "string"}}
        """
        response = client.models.generate_content(model="gemini-2.5-flash", contents=prompt)
        try: out = json.loads(response.text)
        except: out = {"score":0,"reason":"fallback"}
        return out

    def receive(self, msg):
        if msg.get('action')=='rank_jobs':
            user_id = msg.get('user_id')
            profile = memory.get_profile(user_id)
            jobs = memory.list_jobs()
            scored = []
            for j in jobs:
                res = self.score_with_gemini(profile,j)
                j2=dict(j); j2.update(res); scored.append(j2)
            ranked=sorted(scored,key=lambda x:x['score'],reverse=True)
            return {'ranked':ranked}
        return {'error':'unknown action'}

match_agent = MatchAgent()
print("MatchAgent ready.")


**8.Resume & Cover Generator**

In [ ]:
class ResumeGeneratorAgent:
    def generate_with_gemini(self, profile, job, kind='resume'):
        prompt=f"""
        Generate a {kind} for the user.
        Profile JSON: {json.dumps(profile)}
        Job JSON: {json.dumps(job)}
        Return plain text only.
        """
        response=client.models.generate_content(model="gemini-2.5-flash", contents=prompt)
        return response.text.strip()

    def save_docx(self,text,filename):
        doc=Document()
        for line in text.splitlines(): doc.add_paragraph(line)
        doc_path=str(ARTIFACTS_DIR/filename)
        doc.save(doc_path)
        return doc_path

    def receive(self,msg):
        if msg.get('action') in ('generate_resume','generate_cover'):
            profile = msg.get('profile') or memory.get_profile(msg.get('user_id'))
            job = msg.get('job')
            kind='resume' if msg.get('action')=='generate_resume' else 'cover'
            text=self.generate_with_gemini(profile,job,kind=kind)
            filename=f"{profile.get('user_id','user')}_{job['id']}_{kind}.docx"
            path=self.save_docx(text,filename)
            return {'path':path,'text':text}
        return {'error':'unknown action'}

resume_agent = ResumeGeneratorAgent()
print("ResumeGeneratorAgent ready.")


**9.Auto-Apply & Tracker**

In [ ]:
class AutoApplyAgent:
    def auto_apply(self,profile,job):
        app={"user_id":profile['user_id'],"job_id":job['id'],"job_title":job['title'],
             "company":job.get('company','Demo Corp'),"status":"Applied"}
        memory.log_application(app)
        return app
    def receive(self,msg):
        if msg.get('action')=='apply':
            profile=msg.get('profile') or memory.get_profile(msg.get('user_id'))
            job=msg.get('job')
            return {"application":self.auto_apply(profile,job)}
        return {"error":"unknown action"}

class ApplicationTrackerAgent:
    def list_applications(self,user_id=None):
        if user_id: return memory.db['applications'].get(user_id,[])
        return memory.db['applications']
    def receive(self,msg):
        if msg.get('action')=='list_applications':
            user_id=msg.get('user_id')
            return {"applications":self.list_applications(user_id)}
        return {"error":"unknown action"}

auto_apply_agent = AutoApplyAgent()
app_tracker_agent = ApplicationTrackerAgent()
print("AutoApply & Tracker agents ready.")


**10.Demo Run**

In [ ]:
# Demo profile + job search
resume_text="Fazna, BI Analyst, 1 years Python & AI experience"
queries=["Python Developer","AI Engineer"]

# 1️⃣ Extract profile
profile = profile_agent.receive({'action':'extract_profile','resume_text':resume_text})['profile']
print("Profile extracted:", profile)

# 2️⃣ Search jobs
jobs=[]
for q in queries: jobs.extend(search_agent.receive({'action':'search_jobs','query':q,'limit':5})['jobs'])
memory.save_jobs(jobs)
print(f"{len(jobs)} jobs discovered.")

# 3️⃣ Rank
ranked_jobs=match_agent.receive({'action':'rank_jobs','user_id':profile['user_id']})['ranked']
print("Top 5 ranked jobs:")
for j in ranked_jobs[:5]: print("-",j['title'],j['company'],"score:",j['score'])

# 4️⃣ Generate resume & cover, auto-apply to top 2
applied=[]
for job in ranked_jobs[:2]:
    resume_path=resume_agent.receive({'action':'generate_resume','profile':profile,'job':job})['path']
    cover_text=resume_agent.generate_with_gemini(profile,job,kind='cover')
    app_resp=auto_apply_agent.receive({'action':'apply','profile':profile,'job':job})
    applied.append(app_resp['application'])
print("Applied to jobs:",len(applied))

# 5️⃣ List applications
apps=app_tracker_agent.receive({'action':'list_applications','user_id':profile['user_id']})
print("Tracked applications:")
for a in apps['applications']: print("-",a['job_title'],"at",a['company'])
